In [2]:
!pip install pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 11.7 MB 182 kB/s eta 0:00:01    |██████████████████████▊         | 8.3 MB 187 kB/s eta 0:00:19


In [1]:
import sys
import os
import pandas as pd
import json
import random
import shutil

ModuleNotFoundError: No module named 'pandas'

In [99]:
def get_log_path_dict(train,lang):   
    if train == True:
        log_path = os.path.join(os.getcwd(), "output","train",lang)
    else:
        log_path = os.path.join(os.getcwd(), "output","valid",lang)
    print(log_path)
    for root, dirs, files in os.walk(log_path):
        return files

def makedir(new_dir):
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

def split(language_list):
    random.seed(1)

    MAX_EPOCH = 10
    BATCH_SIZE = 2
    LR = 0.001
    log_interval = 10
    val_interval = 1
    print(os.getcwd())
    dataset_dir = os.path.join(os.getcwd(), "data")
    print(dataset_dir)
    split_dir = os.path.join(os.getcwd(), "output")
    train_dir = os.path.join(split_dir, "train")
    valid_dir = os.path.join(split_dir, "valid")
    test_dir = os.path.join(split_dir, "test")

    train_pct = 0.7
    valid_pct = 0.3
    print(dataset_dir)
    # for root, dirs, files in os.walk(dataset_dir):
    #     # dirs = ['german', 'english', 'french', 'spanish']
    for sub_dir in language_list:
        print(sub_dir)
        imgs = os.listdir(os.path.join(dataset_dir, sub_dir))
        imgs = list(filter(lambda x: x.endswith('.wav'), imgs))
        random.shuffle(imgs)
        img_count = len(imgs)

        train_point = int(img_count * train_pct)
        valid_point = int(img_count * (train_pct + valid_pct))

        for i in range(img_count):
            if i < train_point:
                out_dir = os.path.join(train_dir, sub_dir)
            elif i < valid_point:
                out_dir = os.path.join(valid_dir, sub_dir)
            else:
                out_dir = os.path.join(test_dir, sub_dir)

            makedir(out_dir)

            target_path = os.path.join(out_dir, imgs[i])
            src_path = os.path.join(dataset_dir, sub_dir, imgs[i])

            shutil.copy(src_path, target_path)

        print('Class:{}, train:{}, valid:{}, test:{}'.format(sub_dir, train_point, valid_point-train_point,
                                                                img_count-valid_point))

In [100]:
language_list = ['english','french']
split(language_list)

/Users/haoyuhuang/Desktop/umsi/2022_win/545/project
/Users/haoyuhuang/Desktop/umsi/2022_win/545/project/data
/Users/haoyuhuang/Desktop/umsi/2022_win/545/project/data
english
Class:english, train:98, valid:42, test:0
french
Class:french, train:77, valid:33, test:0


In [104]:
train_json_list = []

for lang in language_list:
    file_list = get_log_path_dict(True,lang)
    for i in range(len(file_list)):
        train_json_list.append({'wav':"/root/autodl-tmp/ast/data/sls/audioset/data/audio/train/"+lang+"/" + file_list[i],'labels':lang})
dict = {'data':train_json_list}
# jsonString = json.dumps(dict, indent=4)
with open("sample_train_data.json", "w") as outfile:
    json.dump(dict, outfile)

/Users/haoyuhuang/Desktop/umsi/2022_win/545/project/output/train/english
/Users/haoyuhuang/Desktop/umsi/2022_win/545/project/output/train/french


In [105]:
test_json_list = []
for lang in language_list:
    file_list = get_log_path_dict(False,lang)
    for i in range(len(file_list)):
        test_json_list.append({'wav':"/root/autodl-tmp/ast/data/sls/audioset/data/audio/eval/"+lang+ "/" + file_list[i],'labels':lang})
dict = {'data':test_json_list}
with open("sample_eval_data.json", "w") as outfile:
    json.dump(dict, outfile)

/Users/haoyuhuang/Desktop/umsi/2022_win/545/project/output/valid/english
/Users/haoyuhuang/Desktop/umsi/2022_win/545/project/output/valid/french


In [103]:
label_df = pd.DataFrame({'index':list(range(len(language_list))),'mid':language_list,'display_name':language_list})
label_df.to_csv('class_labels_indices.csv',index=False)